In [1]:
from GBP.data import DataGenerator
from GBP.gbp import run_GaBP_SYNC_ACCELERATED, run_GaBP_HARDWARE_BESTCASE_RESIDUAL, run_GaBP_HARDWARE_BESTCASE, run_GaBP_HARDWARE_ACCELERATED, run_GaBP_HARDWARE_ACCELERATED_RESIDUAL, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS, run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_STREAMS_STOCHASTIC
# , run_GaBP_HARDWARE_ACCELERATED_EXCLUSION_NEIGHBOURS_ACROSS_2_STREAMS
from GBP.utilities import HiddenPrints
from GBP.visulisation import set_plot_options, get_plot_colors, NetworkxGraph, AnalyzeResult
 
import warnings
import matplotlib
import numpy as np
import math
import random

# Option 1: Suppress all warnings
warnings.filterwarnings("ignore")

set_plot_options()
colors = get_plot_colors()

data_gen = DataGenerator()
result_analyzer = AnalyzeResult()

In [2]:
# import random

# def custom_sample(items, n):
#     indices = random.sample(range(len(items)), n)
#     return [items[i] for i in indices]

# # Example usage
# items = ["apple", "banana", "orange", "grape", "kiwi"]
# n = 10
# selected_items = custom_sample(items, n)
# print(selected_items)

In [3]:
num_nodes = 200

sync_convergence_threshold = 1*10**-5

NODE_UPDT_PE =  20
PEs = 1

#### Sync

In [4]:
sum_of_iterations = 0
num_iterations = 1

TRIES = 1000
ASYNC_ITER = 10

for it in range(TRIES):
    A, b = data_gen.get_1D_line_matrix_PSD(num_nodes, scaling=True, normalized=False)
    print(f"---------- ITERATION: {it} ----------")
    # A, b = data_gen.get_2D_lattice_matrix(int(math.sqrt(num_nodes)), int(math.sqrt(num_nodes)))
    A = data_gen.cut_random_edges(A, 0)
    graph = NetworkxGraph(A)
    P_i, mu_i, N_i, P_ii, mu_ii, P_ij, mu_ij, iter_dist, stand_divs, means, iteration = run_GaBP_SYNC_ACCELERATED(A, b, max_iter=100_000, mae=False, convergence_threshold=sync_convergence_threshold, show=True)
    sum_of_iterations += iteration
    final_mean = list(mu_i)
    final_std = P_i
    if iteration < 100000:
        break
    else:
        iteration = float("inf")
        print("=========== RESTART ===========")

print(f"NUMBER SYNC ITERATIONS = {iteration} using {it} tries")

ValueError: low >= high

#### Random - No Cache

In [ ]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Random - with Cache

In [ ]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Random (Exclusion) - No Cache

In [ ]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

it = 0

while (it < num_iterations):

    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration, _ = run_GaBP_HARDWARE_ACCELERATED_EXCLUSION(A, b, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=100_000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    
    if iteration > 0:
        sum_of_iterations += iteration
        print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")
        it += 1

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Random (Exclusion) - With Cache

In [ ]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration, _ = run_GaBP_HARDWARE_ACCELERATED_EXCLUSION(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Residual - with Cache

In [ ]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=False, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Residual - with Cache

In [ ]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=True, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

In [ ]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    # print(f"-------------- ITERATION = {it+1} --------------")
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED_RESIDUAL(A, b, caching=True, capping=1, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    sum_of_iterations += iteration
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

#### Fixed - No Caching

In [ ]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    node_update_schedule = np.arange(num_nodes, dtype=np.int64)
    np.random.shuffle(node_update_schedule)
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=False, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")
    # print(f"streams = {iteration}")
    # print(f"node_update_schedule = {node_update_schedule}")
    sum_of_iterations += iteration

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

### Fixed - Cache

In [ ]:
async_convergence_threshold = 1*10**-2

sum_of_iterations = 0
num_iterations = ASYNC_ITER

for it in range(0,num_iterations):
    node_update_schedule = np.arange(num_nodes, dtype=np.int64)
    np.random.shuffle(node_update_schedule)
    P_i, mu_i, iteration = run_GaBP_HARDWARE_ACCELERATED(A, b, caching=True, mode='fixed', node_update_schedule_enter=node_update_schedule, node_updates_per_pe=NODE_UPDT_PE, number_pes=PEs, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=False)
    print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")
    # print(f"streams = {iteration}")
    # print(f"node_update_schedule = {node_update_schedule}")
    sum_of_iterations += iteration

print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

### NO STREAMS (BESTCASE)

In [ ]:
# async_convergence_threshold = 1*10**-2

# sum_of_iterations = 0
# num_iterations = 100

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_BESTCASE(A, b, node_updates_per_stream=NODE_UPDT_PE, TRUE_MEAN=final_mean, max_iter=10000, mae=False, convergence_threshold=async_convergence_threshold, show=True)
#     sum_of_iterations += iteration
#     print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")

### NO STREAMS (RESIDUAL)

In [ ]:
# async_convergence_threshold = 1*10**-2

# sum_of_iterations = 0
# num_iterations = 100

# for it in range(0,num_iterations):
#     # print(f"-------------- ITERATION = {it+1} --------------")
#     P_i, mu_i, iteration = run_GaBP_HARDWARE_BESTCASE_RESIDUAL(A, b, node_updates_per_stream=NODE_UPDT_PE, TRUE_MEAN=final_mean, max_iter=500, mae=False, convergence_threshold=async_convergence_threshold, show=False)
#     sum_of_iterations += iteration
#     print(f"-------------- ITERATION = {it+1} => Streams = {iteration} --------------")

# print(f"AVE. ASYNC ITERATIONS = {sum_of_iterations/num_iterations}")